In [1]:
from data_source.vectorDatabaseChroma import create_database
%load_ext autoreload
%autoreload 2

In [130]:
chromadb,speakers_list_1,speakers_list_2,speakers_list_3,speakers_list_4,sec_form_names,earnings_call_quarter_vals = create_database("META","2023",embed_type="sentence_transformer")

/home/athekunal/Finance Data/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Earnings Call Q1
Earnings Call Q2
Earnings Call Q3
Earnings Call Q4
SEC
Started Scraping
Scraped
Started Extracting


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Extracted


In [131]:
langchain_chromadb,speakers_list_1,speakers_list_2,speakers_list_3,speakers_list_4,sec_form_names,earnings_call_quarter_vals = create_database("META","2023",embed_type="sentence_transformer")

In [132]:
langchain_chromadb.similarity_search("What is the revenue breakdown of Nvidia?",k=5)

[Document(page_content='was more than 10\xa0times higher than in the Asia-Pacific region. --ARPU:--$53.01--$52.34--$60.57--$48.29---$50.25--$49.13---$58.77---$48.85---$53.53--------ARPU:--$17.23--$16.50--$19.68--$15.35--$15.64---$14.23--$17.29---$15.51---$17.88 Note: Non-advertising revenue includes RL revenue generated from the delivery of consumer hardware products and FoA Other revenue, which consists of net fees we receive from developers using our Payments infrastructure and revenue from various other sources. Our revenue by', metadata={'accessionNumber': '000132680123000093', 'filingDate': '2023-07-27', 'filing_type': '10-Q2', 'reportDate': '2023-06-30', 'sectionName': 'MANAGEMENT_DISCUSSION'}),
 Document(page_content='was more than 10\xa0times higher than in the Asia-Pacific region. --ARPU:--$48.03--$53.01--$52.34--$60.57---$48.29--$50.25---$49.13---$58.77---$48.85--------ARPU:--$15.49--$17.23--$16.50--$19.68--$15.35---$15.64--$14.23--$17.29---$15.51 Note: Non-advertising revenu

In [133]:
chromadb.similarity_search("What is the revenue breakdown of Nvidia?",k=5,filter={
        "filing_type":{"$eq":"10-Q2"}
    })

[Document(page_content='was more than 10\xa0times higher than in the Asia-Pacific region. --ARPU:--$53.01--$52.34--$60.57--$48.29---$50.25--$49.13---$58.77---$48.85---$53.53--------ARPU:--$17.23--$16.50--$19.68--$15.35--$15.64---$14.23--$17.29---$15.51---$17.88 Note: Non-advertising revenue includes RL revenue generated from the delivery of consumer hardware products and FoA Other revenue, which consists of net fees we receive from developers using our Payments infrastructure and revenue from various other sources. Our revenue by', metadata={'accessionNumber': '000132680123000093', 'filingDate': '2023-07-27', 'filing_type': '10-Q2', 'reportDate': '2023-06-30', 'sectionName': 'MANAGEMENT_DISCUSSION'}),
 Document(page_content='have been $1.09\xa0billion and $1.08\xa0billion higher than actual total revenue and advertising revenue, respectively, for the six months ended June\xa030, 2023. Cost of revenue in the three and six months ended June\xa030, 2023 increased $753\xa0million, or 15%, 

In [134]:
q = chromadb.get(
    where={
        "filing_type":{"$eq":"10-Q2"}
    }
)

In [135]:
q['metadatas'][0]

{'accessionNumber': '000132680123000093',
 'filingDate': '2023-07-27',
 'filing_type': '10-Q2',
 'reportDate': '2023-06-30',
 'sectionName': 'MANAGEMENT_DISCUSSION'}

In [136]:
quarter_speaker_dict = {
    "Q1":speakers_list_1,
    "Q2":speakers_list_2,
    "Q3":speakers_list_3,
    "Q4":speakers_list_4
}

In [137]:
from typing import Annotated
def query_database_earnings_call(
        question: str,
        quarter: str
)->str:
    """This tool will query the earnings call transcripts database for a given question and quarter and it will retrieve
       the relevant text along from the earnings call and the speaker who addressed the relevant documents. This tool helps in answering questions
       from the earnings call transcripts.

    Args:
        question (str): _description_. Question to query the database for relevant documents.
        quarter (str): _description_. the financial quarter that is discussed in the question and possible options are Q1, Q2, Q3, Q4

    Returns:
        str: relevant text along from the earnings call and the speaker who addressed the relevant documents
    """
    assert quarter in earnings_call_quarter_vals, "The quarter should be from Q1, Q2, Q3, Q4"
    
    req_speaker_list = []
    quarter_speaker_list = quarter_speaker_dict[quarter]

    for sl in quarter_speaker_list:
        if sl in question or sl.lower() in question:
            req_speaker_list.append(sl)
    if len(req_speaker_list) == 0:
        req_speaker_list = quarter_speaker_list
    
    relevant_docs = langchain_chromadb.similarity_search(
        question,
        k=5,
        filter={
            "$and":[
                {
                    "quarter":{"$eq":quarter}
                },
                {
                    "speaker":{"$in":req_speaker_list}
                }
            ]
        }
    )

    speaker_releavnt_dict = {}
    for doc in relevant_docs:
        speaker = doc.metadata['speaker']
        speaker_text = doc.page_content
        if speaker not in speaker_releavnt_dict:
            speaker_releavnt_dict[speaker] = speaker_text
        else:
            speaker_releavnt_dict[speaker] += " "+speaker_text
    
    relevant_speaker_text = ""
    for speaker, text in speaker_releavnt_dict.items():
        relevant_speaker_text += speaker + ": "
        relevant_speaker_text += text + "\n\n"

    return relevant_speaker_text


In [138]:
val = query_database_earnings_call(
    "Revenue breakdown",
    "Q4"
)

In [139]:
print(val)

Susan Li: Thanks, Mark, and good afternoon, everyone. Let's begin with our consolidated results. All comparisons are on a year-over-year basis, unless otherwise noted. Q4 total revenue was $40.1 billion, up 25% or 22% on a constant currency basis. Q4 total expenses were $23.7 billion, down 8% compared to last year.  In terms of the specific line items, cost of revenue decreased 8%, 0driven mainly by lower restructuring costs that were partially offset by higher infrastructure-related costs. R&D increased 8% driven the fourth quarter with over 67,300 employees, down 22% from a year ago while up 2% from Q3 as our hiring efforts have resumed. Fourth quarter operating income was $16.4 billion, representing a 41% operating margin. Our tax rate for the quarter was 17%.  Net income was $14 billion or $5.33 per share. Capital expenditures, including principal payments on finance leases, were $7.9 billion, driven by investments in servers, data centers and network infrastructure. Free cash flow

In [140]:
def query_database_sec(
        question: str,
        sec_form_name: str
)->str:
    """This tool will query the SEC Filings database for a given question and form name, and it will retrieve
       the relevant text along from the SEC filings and the section names. This tool helps in answering questions
       from the sec filings.

    Args:
        question (str): _description_. Question to query the database for relevant documents
        sec_form_name (str): _description_. SEC FORM NAME that the question is talking about. It can be 10-K for yearly data and 10-Q for quarterly data. For quarterly data, it can be 10-Q2 to represent Quarter 2 and similarly for other quarters.

    Returns:
        str: Relevant context for the question from the sec filings
    """
    assert sec_form_name in sec_form_names, f'The search form type should be in {sec_form_names}'
    
    relevant_docs = langchain_chromadb.similarity_search(
        question,
        k=5,
        filter={
            "filing_type":{"$eq":sec_form_name}
        }
    )

    relevant_section_dict = {}
    for doc in relevant_docs:
        section = doc.metadata['sectionName']
        section_text = doc.page_content
        if section not in relevant_section_dict:
            relevant_section_dict[section] = section_text
        else:
            relevant_section_dict[section] += " "+section_text
    
    relevant_section_text = ""
    for section, text in relevant_section_dict.items():
        relevant_section_text += section + ": "
        relevant_section_text += text + "\n\n"

    return relevant_section_text

In [141]:
query_database_sec(
    "What is the revenue breakdown of Nvidia?",
    "10-K"
)

'MANAGEMENT_DISCUSSION: was more than 11\xa0times higher than in the Asia-Pacific region. --ARPU:--$60.57--$48.29--$50.25--$49.13---$58.77--$48.85---$53.53---$56.11---$68.44--------ARPU:--$19.68--$15.35--$15.64--$14.23--$17.29---$15.51--$17.88---$19.04---$23.14 Note: Non-advertising revenue includes RL revenue generated from the delivery of consumer hardware products and FoA Other revenue, which consists of revenue from WhatsApp Business Platform, net fees we receive from developers using our Payments infrastructure, and revenue\n\nMANAGEMENT: was more than 11\xa0times higher than in the Asia-Pacific region. --ARPU:--$60.57--$48.29--$50.25--$49.13---$58.77--$48.85---$53.53---$56.11---$68.44--------ARPU:--$19.68--$15.35--$15.64--$14.23--$17.29---$15.51--$17.88---$19.04---$23.14 Note: Non-advertising revenue includes RL revenue generated from the delivery of consumer hardware products and FoA Other revenue, which consists of revenue from WhatsApp Business Platform, net fees we receive fr

In [142]:
from FinCrewAI.books_db import load_database

books_collection = load_database()

def query_database_books(
         question: str
)->str:
    """This tool will query the financial books database for a given question and it will retrieve the relevant text.
        This tool helps in answering questions from the earnings call transcripts.

    Args:
        question (str): _description_. Question to query the database for relevant documents

    Returns:
        str: relevant documents from the financial books database
    """
    relevant_docs = books_collection.similarity_search(
        question,
        k=5,
    )
    releavnt_text = ""
    for doc in relevant_docs:
        releavnt_text += doc.page_content
        releavnt_text += "\n\n"
    return releavnt_text

## AUTOGEN AGENT

In [143]:
from autogen import ConversableAgent, AssistantAgent
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv(),override=True)
llm_config = {"model":"gpt-4-turbo"}

In [144]:
sec_form_system_msg = ""

for sec_form in sec_form_names:
    if sec_form == "10-K":
        sec_form_system_msg+= "10-K for yearly data, "
    elif "10-Q" in sec_form:
        quarter = sec_form[-1]
        sec_form_system_msg+= f"{sec_form} for Q{quarter} data, "
sec_form_system_msg[:-2]

'10-K for yearly data, 10-Q3 for Q3 data, 10-Q2 for Q2 data, 10-Q1 for Q1 data'

In [145]:
earnings_call_system_message = ", ".join(earnings_call_quarter_vals)

In [173]:
system_msg = f"""You are a helpful financial assistant and your task is to select the sec_filings or earnings_call or financial_books to best answer the question. 
You can call to query_database_sec(question,sec_form) by passing question and relevant sec_form names like {sec_form_system_msg} 
or you can call to query_database_earnings_call(question,quarter) by passing question and relevant quarter names with possible values {earnings_call_system_message} 
or you can call to query_database_books(question) to get relevant documents from financial textbooks about valuation and investing philosophies. When you are ready to end the coversation, reply TERMINATE"""
print(system_msg)

You are a helpful financial assistant and your task is to select the sec_filings or earnings_call or financial_books to best answer the question. 
You can call to query_database_sec(question,sec_form) by passing question and relevant sec_form names like 10-K for yearly data, 10-Q3 for Q3 data, 10-Q2 for Q2 data, 10-Q1 for Q1 data,  
or you can call to query_database_earnings_call(question,quarter) by passing question and relevant quarter names with possible values Q1, Q2, Q3, Q4 
or you can call to query_database_books(question) to get relevant documents from financial textbooks about valuation and investing philosophies. When you are ready to end the coversation, reply TERMINATE


In [174]:
user_proxy = ConversableAgent(
    name = "Planner Admin",
    system_message=system_msg,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
)

In [175]:
print(user_proxy.system_message)

You are a helpful financial assistant and your task is to select the sec_filings or earnings_call or financial_books to best answer the question. 
You can call to query_database_sec(question,sec_form) by passing question and relevant sec_form names like 10-K for yearly data, 10-Q3 for Q3 data, 10-Q2 for Q2 data, 10-Q1 for Q1 data,  
or you can call to query_database_earnings_call(question,quarter) by passing question and relevant quarter names with possible values Q1, Q2, Q3, Q4 
or you can call to query_database_books(question) to get relevant documents from financial textbooks about valuation and investing philosophies. When you are ready to end the coversation, reply TERMINATE


In [176]:
tool_proxy = ConversableAgent(
    name="Tool Proxy",
    llm_config=False,
    default_auto_reply="Please select the right database.",
    human_input_mode="NEVER",
)

In [177]:
from autogen import register_function

tools_dict = {
    "sec":[query_database_sec,"Tool to query SEC filings database"],
    "earnings_call": [query_database_earnings_call, "Tool to query earnings call transcripts database"],
    "books": [query_database_books, "Tool to query books database"], 
}

for tool_name,tool in tools_dict.items():
    register_function(
        tool[0],
        caller=user_proxy,
        executor=tool_proxy,
        name = tool[0].__name__,
        description=tool[1]       
    )

In [178]:
user_proxy.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Tool to query SEC filings database',
   'name': 'query_database_sec',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string', 'description': 'question'},
     'sec_form_name': {'type': 'string', 'description': 'sec_form_name'}},
    'required': ['question', 'sec_form_name']}}},
 {'type': 'function',
  'function': {'description': 'Tool to query earnings call transcripts database',
   'name': 'query_database_earnings_call',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string', 'description': 'question'},
     'quarter': {'type': 'string', 'description': 'quarter'}},
    'required': ['question', 'quarter']}}},
 {'type': 'function',
  'function': {'description': 'Tool to query books database',
   'name': 'query_database_books',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string', 'description': 'question'}},
    'required': ['question']}}}]

In [179]:
chat_result = user_proxy.initiate_chat(
    recipient=tool_proxy,
    message="What is the revenue breakdown of Meta for the fiscal year 2023?",
    max_turns=3
)

Planner Admin (to Tool Proxy):

What is the revenue breakdown of Meta for the fiscal year 2023?

--------------------------------------------------------------------------------
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------


Planner Admin (to Tool Proxy):

***** Suggested tool call (call_dZ47xsNPw3SCq0GOfuVEiRSp): query_database_sec *****
Arguments: 
{"question":"What is the revenue breakdown of Meta for the fiscal year 2023?","sec_form_name":"10-K"}
***********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION query_database_sec...
Tool Proxy (to Planner Admin):

Tool Proxy (to Planner Admin):

***** Response from calling tool (call_dZ47xsNPw3SCq0GOfuVEiRSp) *****
FINANCIAL_STATEMENTS: and data used to determine the amount of the federal tax liability recognized and tested the mathematical accuracy of the underlying data and calculations. We also evaluated the appropriateness of the related disclosures included in Note 15 to the consolidated financial statements in relation to these matters. We have served as the Company's auditor since 2007. We have audited Meta Platforms, 

In [180]:
chat_result.chat_history[-4]

{'tool_calls': [{'id': 'call_dZ47xsNPw3SCq0GOfuVEiRSp',
   'function': {'arguments': '{"question":"What is the revenue breakdown of Meta for the fiscal year 2023?","sec_form_name":"10-K"}',
    'name': 'query_database_sec'},
   'type': 'function'}],
 'content': None,
 'role': 'assistant'}

In [181]:
chat_result = user_proxy.initiate_chat(
    recipient=tool_proxy,
    message="What was the spending on AI by Meta?",
    max_turns=5
)

Planner Admin (to Tool Proxy):

What was the spending on AI by Meta?

--------------------------------------------------------------------------------
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------


Planner Admin (to Tool Proxy):

To find specific financial details about Meta's spending on AI, I can query the SEC filings database for their latest annual or quarterly report. Which report would you prefer to check? The most recent annual (10-K) or a specific quarterly (10-Q1, 10-Q2, 10-Q3) report?

--------------------------------------------------------------------------------
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

***** Suggested tool call (call_NSXBztaESerVKRnYdt155ppp): query_database_sec *****
Arguments: 
{"question":"What was the spending on AI by Meta?","sec_form_name":"10-K"}
***********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION query_database_sec...
Tool Proxy (to Planner Admin):

Tool Proxy (to P

In [182]:
chat_result = user_proxy.initiate_chat(
    recipient=tool_proxy,
    message="What are the dark sides of valuation of companies?",
    max_turns=5
)

Planner Admin (to Tool Proxy):

What are the dark sides of valuation of companies?

--------------------------------------------------------------------------------
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

***** Suggested tool call (call_hxfKeMPJ7XuJ933YBJTPvS5s): query_database_books *****
Arguments: 
{"question":"What are the dark sides of valuation of companies?"}
*************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION query_database_books...
Tool Proxy (to Planner Admin):

Tool Proxy (to Planner Admin):

***** Response from calling tool (call_hxfKeMPJ7XuJ933YBJTPvS5s) *****
T hroughout this book, we have emphasized the importance of first principles in valuation and how they should guide us when we're faced

In [183]:
chat_result = user_proxy.initiate_chat(
    recipient=tool_proxy,
    message="What did Mark Zuckerberg say about the future investments of the company?",
    max_turns=5
)

Planner Admin (to Tool Proxy):

What did Mark Zuckerberg say about the future investments of the company?

--------------------------------------------------------------------------------
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

To find out what Mark Zuckerberg said about future investments of the company, I will check the most recent earnings call. This is typically where executives discuss strategies and future investment plans.
***** Suggested tool call (call_nVIOQ6Doe95EZaXZJR8Bb8ms): query_database_earnings_call *****
Arguments: 
{"question":"What did Mark Zuckerberg say about the future investments of the company?","quarter":"Q3"}
*********************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION query_datab

In [189]:
for ct in chat_result.chat_history:
    if ct['role'] == 'assistant':
        print(ct['content'])

What did Mark Zuckerberg say about the future investments of the company?
To find out what Mark Zuckerberg said about future investments of the company, I will check the most recent earnings call. This is typically where executives discuss strategies and future investment plans.
Mark Zuckerberg discussed several key points about the future investments of the company during the Q3 earnings call:

1. **Efficiency and Innovation:** Zuckerberg highlighted the company's progress in becoming a leaner organization that is shipping faster and advancing in its long-term initiatives.

2. **Financial Discipline:** He noted that the company has achieved its highest operating margin in two years, indicating a focus on maintaining financial discipline while continuing to invest heavily for the future.

3. **Future Vision:** Zuckerberg expressed a commitment to continuing work on hardware and computing efficiency. He acknowledged that they are still in the early stages of understanding the economic m

In [192]:

chat_result.chat_history[2]

{'content': 'To find out what Mark Zuckerberg said about future investments of the company, I will check the most recent earnings call. This is typically where executives discuss strategies and future investment plans.',
 'tool_calls': [{'id': 'call_nVIOQ6Doe95EZaXZJR8Bb8ms',
   'function': {'arguments': '{"question":"What did Mark Zuckerberg say about the future investments of the company?","quarter":"Q3"}',
    'name': 'query_database_earnings_call'},
   'type': 'function'}],
 'role': 'assistant'}